In [ ]:
# Remove any generated glue code
!find . -mindepth 1 ! \( -name "airfoil.ipynb" -o -name "EulerSolver.cpp" -o -name "exahype.log-filter" \) -exec rm -rf {} +

In [ ]:
import peano4, exahype2

domain_size = [120.0,  120.0]
domain_offset = [-10.0, -10.0]
min_depth = 5
max_depth = 0  # AMR levels

unknowns = { "p": 1, "v": 2, "E": 1 }
auxiliary_variables = {}
max_h = 1.1 * min(domain_size) / (3.0 ** min_depth)
min_h = max_h / (3.0 ** max_depth)

my_solver = exahype2.solvers.fv.rusanov.GlobalAdaptiveTimeStep(
    name                  = "EulerSolver",
    patch_size            = 22,
    min_volume_h          = min_h,
    max_volume_h          = max_h,
    time_step_relaxation  = 0.1,
    unknowns              = unknowns,
    auxiliary_variables   = auxiliary_variables
)

my_solver.plot_description = ", ".join(unknowns.keys())

my_solver.set_implementation(
    initial_conditions    = exahype2.solvers.PDETerms.User_Defined_Implementation,
    boundary_conditions   = exahype2.solvers.PDETerms.User_Defined_Implementation,
    eigenvalues           = exahype2.solvers.PDETerms.User_Defined_Implementation,
    flux                  = exahype2.solvers.PDETerms.User_Defined_Implementation,
    # ncp                 = exahype2.solvers.PDETerms.User_Defined_Implementation
)

exahype2_project = exahype2.Project(
    namespace     = ["exahype2", "training", "euler"],
    directory     = ".",
    project_name  = "Airfoil",
    executable    = "Airfoil"
)

exahype2_project.add_solver(my_solver)
exahype2_project.set_output_path("solutions")

exahype2_project.set_global_simulation_parameters(
    dimensions            = 2,
    size                  = domain_size,
    offset                = domain_offset,
    min_end_time          = 10.,
    max_end_time          = 10.,
    first_plot_time_stamp = 0.0,
    time_in_between_plots = 0.5,
    periodic_BC           = [False, False],
)

exahype2_project.set_load_balancing(
    "toolbox::loadbalancing::strategies::SpreadOutOnceGridStagnates", # SpreadOut, SpreadOutHierarchically
    "new ::exahype2::LoadBalancingConfiguration()"
)

exahype2_project.set_Peano4_installation("", peano4.output.CompileMode.Release)
peano4_project = exahype2_project.generate_Peano4_project()
peano4_project.output.makefile.add_CXX_flag("-w")
peano4_project.output.makefile.add_CXX_flag("-O3")
peano4_project.output.makefile.add_CXX_flag("-DNDEBUG")
peano4_project.output.makefile.add_CXX_flag("-DPeanoDebug=0")
peano4_project.generate()
peano4_project.build(make=True, make_clean_first=False, throw_away_data_after_build=False)

In [ ]:
!OMP_NUM_THREADS=1 ./Airfoil > output.txt

!grep "initial grid construction" output.txt | head -n 1
!grep "plotting" output.txt | head -n 1
!grep "time stepping" output.txt | head -n 1
!grep "average time per mesh sweep" output.txt | head -n 1

In [ ]:
!python $PEANO_ROOT/python/peano4/visualisation/render.py solutions/solution-EulerSolver.peano-patch-file